In [2]:
pip install -r  requirements.txt -q


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
from PIL import Image

base_path = "/Users/karlgustav/Documents/GitHub/study/master-thesis/server/src/research/"
img1_path = f"{base_path}data/img1.png"
img2_path = f"{base_path}data/img2.png"
img3_path = f"{base_path}data/img3.png"
img4_path = f"{base_path}data/img4.png"

img1 = np.array(Image.open(img1_path))
img2 = np.array(Image.open(img2_path))
img3 = np.array(Image.open(img3_path))
img4 = np.array(Image.open(img4_path))

In [40]:
import numpy as np
def euclidean_distance(a: np.array, b: np.array, allow_not_equal=False) -> float:
    if a.shape == b.shape:
        return np.linalg.norm(a - b)
    if allow_not_equal is False:
        raise ValueError(f"Images must have the same shape. a.shape: {a.shape}, b.shape: {b.shape}")

    a_number_of_rows, a_number_of_col = a.shape
    b_number_of_rows, b_number_of_col = b.shape

    rows, columns = min(a_number_of_rows, b_number_of_rows), min(a_number_of_col, b_number_of_col)

    if a_number_of_rows != rows or a_number_of_col != columns:
        a = np.resize(a, (rows, columns))
    if b_number_of_rows != rows or b_number_of_col != columns:
        b = np.resize(b, (rows, columns))

    return euclidean_distance(a, b)

In [49]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

# Load ResNet-50 model pre-trained on ImageNet
blackbox_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

def convert_to_rgb(image):
    # If the image has an alpha channel, remove it to convert to RGB
    if image.shape[-1] == 4:
        image = image[..., :3]
    return image

def preprocess_images(img_array):
    img_array = np.array([tf.image.resize(img_to_array(convert_to_rgb(img)), (224, 224)) for img in img_array])
    return preprocess_input(img_array)

def get_image_embedding(images):
    return blackbox_model.predict(images)

array = preprocess_images([img1,img2,img3,img4])
embeddings = get_image_embedding(array)

1/1 [==============================] - 0s 438ms/step


Analysis

In [53]:
imgnr_img = {1:img1,2:img2,3:img3,4:img4}
img_pairs = {1:[1,2,3,4],2:[1,2,3,4],3:[1,2,3,4],4:[1,2,3,4]}

In [54]:
def raw_distance_results():
    for img_nr,pairs in img_pairs.items():
        my_img = imgnr_img[img_nr]
        for pair_nr in pairs:
            my_pair = imgnr_img[pair_nr]
            dist1 = euclidean_distance(my_img[0],my_pair[0],allow_not_equal=True)
            print("Euclidian distance between image "+str(img_nr)+" and image "+str(pair_nr)+" is "+str(round(dist1,0)))
            embedding1 = embeddings[img_nr-1]
            embedding2 = embeddings[pair_nr-1]
            dist2 = euclidean_distance(embedding1 ,embedding2)
            print("Euclidian distance between image "+str(img_nr)+" embedding and image "+str(pair_nr)+" embedding is "+str(round(dist2,0)))
raw_distance_results()

Euclidian distance between image 1 and image 1 is 0.0
Euclidian distance between image 1 embedding and image 1 embedding is 0.0
Euclidian distance between image 1 and image 2 is 6410.0
Euclidian distance between image 1 embedding and image 2 embedding is 572.0
Euclidian distance between image 1 and image 3 is 6786.0
Euclidian distance between image 1 embedding and image 3 embedding is 510.0
Euclidian distance between image 1 and image 4 is 1479.0
Euclidian distance between image 1 embedding and image 4 embedding is 586.0
Euclidian distance between image 2 and image 1 is 2337.0
Euclidian distance between image 2 embedding and image 1 embedding is 572.0
Euclidian distance between image 2 and image 2 is 0.0
Euclidian distance between image 2 embedding and image 2 embedding is 0.0
Euclidian distance between image 2 and image 3 is 0.0
Euclidian distance between image 2 embedding and image 3 embedding is 557.0
Euclidian distance between image 2 and image 4 is 3172.0
Euclidian distance betwee

In [60]:
def closest_images(img_nr):
    my_img = imgnr_img[img_nr]
    pairs = img_pairs[img_nr]
    
    imgnr_distance_img = {}
    imgnr_distance_img_emb = {}
        
    for pair_nr in pairs:
        my_pair = imgnr_img[pair_nr]
        dist1 = euclidean_distance(my_img[0],my_pair[0],allow_not_equal=True)
        imgnr_distance_img[pair_nr] = round(dist1)
        
        embedding1 = embeddings[img_nr-1]
        embedding2 = embeddings[pair_nr-1]
        dist2 = euclidean_distance(embedding1 ,embedding2)
        imgnr_distance_img_emb[pair_nr] = dist2
        
    print("Closest using image euclidian disntances")
    print(imgnr_distance_img)
    print("Closest using image embedding euclidian disntances")
    print(imgnr_distance_img_emb)

closest_images(1)

Closest using image euclidian disntances
{1: 0, 2: 6410, 3: 6786, 4: 1479}
Closest using image embedding euclidian disntances
{1: 0.0, 2: 572.1848, 3: 509.5003, 4: 585.7291}
